In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential

In [2]:
class InvariantModule(tf.keras.Model):
    """Implements an invariant module with keras."""
    
    def __init__(self, meta):
        super(InvariantModule, self).__init__()
        
        self.s1 = Sequential([Dense(**meta['dense_s1_args']) for _ in range(meta['n_dense_s1'])])
        self.s2 = Sequential([Dense(**meta['dense_s2_args']) for _ in range(meta['n_dense_s2'])])
                    
    def call(self, x):
        """Performs the forward pass of a learnable invariant transform.
        
        Parameters
        ----------
        x : tf.Tensor
            Input of shape (batch_size, N, x_dim)
        
        Returns
        -------
        out : tf.Tensor
            Output of shape (batch_size, out_dim)
        """
        
        x_reduced = tf.reduce_mean(self.s1(x), axis=1)
        print(x_reduced.shape)
        out = self.s2(x_reduced)
        return out
    
    
class EquivariantModule(tf.keras.Model):
    """Implements an equivariant module with keras."""
    
    def __init__(self, meta):
        super(EquivariantModule, self).__init__()
        
        self.invariant_module = InvariantModule(meta)
        self.s3 = Sequential([Dense(**meta['dense_s3_args']) for _ in range(meta['n_dense_s3'])])
                    
    def call(self, x):
        """Performs the forward pass of a learnable equivariant transform.
        
        Parameters
        ----------
        x : tf.Tensor
            Input of shape (batch_size, N, x_dim)
        
        Returns
        -------
        out : tf.Tensor
            Output of shape (batch_size, N, equiv_dim)
        """
        
        # Store N
        N = int(x.shape[1])
        print('Shape x')
        print(x.shape)
        # Output dim is (batch_size, inv_dim) - > (batch_size, N, inv_dim)
        out_inv = self.invariant_module(x)
        print('Shape 1')
        print(out_inv.shape)
        out_inv_rep = tf.stack([out_inv] * N, axis=1)
        print('Shape 2')
        print(out_inv_rep.shape)
        
        # Concatenate each x with the repeated invariant embedding
        out_c = tf.concat([x, out_inv_rep], axis=-1)
        print(out_c.shape)
        # Pass through equivariant func
        out = self.s3(out_c)
        print(out.shape)
        return out


class InvariantNetwork(tf.keras.Model):
    """Implements an invariant network with keras.
    """

    def __init__(self, meta={}):
        super(InvariantNetwork, self).__init__()

        
        self.equiv_seq = Sequential([EquivariantModule(meta) for _ in range(meta['n_equiv'])])
        print(self.equiv_seq)
        self.inv = InvariantModule(meta)
    
    def call(self, x):
        """ Performs the forward pass of a learnable deep invariant transformation consisting of
        a sequence of equivariant transforms followed by an invariant transform.
        
        Parameters
        ----------
        x : tf.Tensor
            Input of shape (batch_size, n_obs, data_dim)
        
        Returns
        -------
        out : tf.Tensor
            Output of shape (batch_size, out_dim + 1)
        """
        
        # Extract n_obs and create sqrt(N) vector
        N = int(x.shape[1])
        N_rep = tf.math.sqrt(N * tf.ones((x.shape[0], 1)))

        # Pass through series of augmented equivariant transforms
        out_equiv = self.equiv_seq(x)

        # Pass through final invariant layer and concatenate with N_rep
        out_inv = self.inv(out_equiv)
        out = tf.concat((out_inv, N_rep), axis=-1)

        return out
    

In [3]:
meta_dict={
        'n_dense_s1': 2,
        'n_dense_s2': 2,
        'n_dense_s3': 2,
        'n_equiv':    2,
        'dense_s1_args': {'activation': 'relu', 'units': 32},
        'dense_s2_args': {'activation': 'relu', 'units': 64},
        'dense_s3_args': {'activation': 'relu', 'units': 32}
    }

In [4]:
inv = InvariantNetwork(meta=meta_dict)

In [5]:
noise = np.random.random((10000,1,1024))

In [6]:
inv.call(noise)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Shape x
(10000, 1, 1024)
(10000, 32)
Shape 1
(10000, 64)
Shape 2
(10000, 1, 64)
(10000, 1, 1088)
(10000, 1, 32)
Shape x
(10000, 1, 32)
(10000, 32)
Shape 1
(10000, 64)
Shape 2
(10000, 1, 64)
(10000, 1, 96)
(10000, 1, 32)
(10000, 32)


<tf.Tensor: shape=(10000, 65), dtype=float32, numpy=
array([[0.01622252, 0.00672532, 0.        , ..., 0.00717338, 0.        ,
        0.99999994],
       [0.00717196, 0.00704909, 0.        , ..., 0.        , 0.        ,
        0.99999994],
       [0.0177478 , 0.01310369, 0.01747869, ..., 0.06549501, 0.        ,
        0.99999994],
       ...,
       [0.00623761, 0.01214561, 0.        , ..., 0.        , 0.        ,
        0.99999994],
       [0.01501398, 0.0100764 , 0.00240402, ..., 0.00429376, 0.        ,
        0.99999994],
       [0.00836204, 0.00905464, 0.        , ..., 0.        , 0.        ,
        0.99999994]], dtype=float32)>

In [7]:
test1 = np.array([[3,4],[5,6]])
tf.stack([test1] * 3, 1)

<tf.Tensor: shape=(2, 3, 2), dtype=int64, numpy=
array([[[3, 4],
        [3, 4],
        [3, 4]],

       [[5, 6],
        [5, 6],
        [5, 6]]])>

In [8]:
number_presimulated = 1000
theta_presimulated = prior.sample((number_presimulated,))
x_presimulated = simulator(theta_presimulated)

print('Shape of x_presimulated    : ', x_presimulated.shape)

NameError: name 'prior' is not defined

In [9]:
inference = SNPE(prior)
inference.provide_presimulated(theta_presimulated, x_presimulated, from_round=0)

NameError: name 'SNPE' is not defined

In [10]:
help(inference)

NameError: name 'inference' is not defined